In [ ]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath("")))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

In [ ]:
# Find all group makeup tags
stash_tags = stash_client.get_tags_by_names(
    [
        "Group Makeup Generic",
        "Group Makeup Specific",
        "Full Movie",
        "Compilation",
        "Multiple Sex Scenes in a Scene",
        "Behind the Scenes",
        "TV Series",
        "Non-Sex Performer",
        "Virtual Sex",
        "Missing Performer (Male)",
        "Missing Performer (Female)",
        "Data Quality: Multiple Generic Group Makeup Tags",
        "Data Quality: Multiple Specific Group Makeup Tags",
    ]
)

excluded_tags = [
    stash_tags.full_movie["id"],
    stash_tags.compilation["id"],
    stash_tags.multiple_sex_scenes_in_a_scene["id"],
    stash_tags.behind_the_scenes["id"],
    stash_tags.tv_series["id"],
    stash_tags.non_sex_performer["id"],
    stash_tags.virtual_sex["id"],
    stash_tags.missing_performer_male["id"],
    stash_tags.missing_performer_female["id"],
]

group_makeup_generic_tags = stash.find_tags(
    {
        "parents": {
            "value": [stash_tags.group_makeup_generic["id"]],
            "modifier": "INCLUDES",
            "depth": -1,
        }
    },
    fragment="id name",
)
group_makeup_specific_tags = stash.find_tags(
    {
        "parents": {
            "value": [stash_tags.group_makeup_specific["id"]],
            "modifier": "INCLUDES",
            "depth": -1,
        }
    },
    fragment="id name",
)

# Reset issue tags

In [ ]:
scenes_with_issue_tags = stash.find_scenes({
    "tags": {
        "value": [stash_tags.data_quality_multiple_generic_group_makeup_tags["id"], stash_tags.data_quality_multiple_specific_group_makeup_tags["id"]],
        "modifier": "INCLUDES",
        "depth": 0,
        "excludes": excluded_tags
    }
}, fragment="id title tags { id name }")

scene_ids = [scene["id"] for scene in scenes_with_issue_tags]
stash_client.bulk_scene_update(scene_ids, [stash_tags.data_quality_multiple_generic_group_makeup_tags["id"], stash_tags.data_quality_multiple_specific_group_makeup_tags["id"]], "REMOVE")

# Find all scenes

In [ ]:
# Find all scenes with group makeup tags
scenes_with_group_makeup_tags = stash.find_scenes({
    "tags": {
        "value": [stash_tags.group_makeup_generic["id"], stash_tags.group_makeup_specific["id"]],
        "modifier": "INCLUDES",
        "depth": -1,
        "excludes": excluded_tags
    }
}, fragment="id title tags { id name }")

# Create a mapping of all group makeup tag IDs to names
all_group_makeup_tags = group_makeup_generic_tags + group_makeup_specific_tags
tag_id_to_name = {tag["id"]: tag["name"] for tag in all_group_makeup_tags}
all_group_makeup_tag_ids = set(tag_id_to_name.keys())

print(f"📊 Found {len(scenes_with_group_makeup_tags)} scenes with group makeup tags")
print(f"📊 Found {len(group_makeup_generic_tags)} generic group makeup tags")
print(f"📊 Found {len(group_makeup_specific_tags)} specific group makeup tags")

# Generic group makeup tags

In [ ]:
# Extract generic group makeup tag IDs
generic_group_makeup_tag_ids = [tag["id"] for tag in group_makeup_generic_tags]

print(f"📊 Found {len(generic_group_makeup_tag_ids)} generic group makeup tags")

# Convert scenes to DataFrame for easier analysis
scenes_data = []
for scene in scenes_with_group_makeup_tags:
    scene_tags = [tag["id"] for tag in scene["tags"]]
    # Count how many generic group makeup tags this scene has
    generic_tags_count = sum(1 for tag_id in scene_tags if tag_id in generic_group_makeup_tag_ids)
    
    # Get only the group makeup tag names for this scene
    group_makeup_tag_names = [
        tag_id_to_name[tag_id] 
        for tag_id in scene_tags 
        if tag_id in all_group_makeup_tag_ids
    ]
    
    scenes_data.append({
        "scene_id": scene["id"],
        "title": scene["title"],
        "generic_tags_count": generic_tags_count,
        "group_makeup_tags": group_makeup_tag_names
    })

df_scenes = pl.DataFrame(scenes_data)
df_scenes


In [ ]:
# Find scenes with more than one generic group makeup tag
multiple_generic_tags_scenes = df_scenes.filter(pl.col("generic_tags_count") > 1)

print(f"🎯 Found {len(multiple_generic_tags_scenes)} scenes with more than one generic group makeup tag")
print(f"📈 Out of {len(df_scenes)} total scenes with group makeup tags")

# Show the distribution of generic tag counts
tag_count_distribution = df_scenes.group_by("generic_tags_count").agg(
    pl.len().alias("scene_count")
).sort("generic_tags_count")

print("\n📊 Distribution of generic group makeup tag counts:")
print(tag_count_distribution)

# Display the scenes with multiple generic tags
if len(multiple_generic_tags_scenes) > 0:
    print(f"\n🔍 Scenes with multiple generic group makeup tags:")
    multiple_generic_tags_scenes.select(["scene_id", "title", "generic_tags_count"])


In [ ]:
# Show detailed tags for scenes with multiple generic group makeup tags
if len(multiple_generic_tags_scenes) > 0:
    print("🔍 Detailed analysis of scenes with multiple generic group makeup tags:")
    print("="*80)
    
    # Create a mapping of tag names to identify which are generic
    generic_tag_names = [tag["name"] for tag in group_makeup_generic_tags]
    
    for scene in multiple_generic_tags_scenes.iter_rows(named=True):
        scene_id = scene["scene_id"]
        title = scene["title"]
        group_makeup_tags = scene["group_makeup_tags"]
        
        # Find which generic tags this scene has
        generic_tags_for_scene = [tag_name for tag_name in group_makeup_tags if tag_name in generic_tag_names]
        
        print(f"\n📽️  Scene ID: {scene_id}")
        print(f"📋 Title: {title}")
        print(f"🏷️  All Group Makeup Tags ({len(group_makeup_tags)}):")
        for tag_name in group_makeup_tags:
            is_generic = tag_name in generic_tag_names
            prefix = "   🔵" if is_generic else "   🟡"
            print(f"{prefix} {tag_name}")
        print(f"🏷️  Generic Group Makeup Tags ({len(generic_tags_for_scene)}):")
        for tag_name in generic_tags_for_scene:
            print(f"   • {tag_name}")
        print("-" * 80)
        
else:
    print("✅ No scenes found with multiple generic group makeup tags!")


In [ ]:
# Apply the specific issue tag to scenes with multiple specific group makeup tags
if len(multiple_generic_tags_scenes) > 0:
    scene_ids_generic = multiple_generic_tags_scenes["scene_id"].to_list()
    result_generic = stash_client.bulk_scene_update(
        scene_ids_generic, 
        [stash_tags.data_quality_multiple_generic_group_makeup_tags["id"]], 
        "ADD"
    )
    print(f"✅ Applied 'Data Quality: Multiple Generic Group Makeup Tags' to {len(scene_ids_generic)} scenes")
    result_generic
else:
    print("✅ No scenes need the generic group makeup tags data quality tag")


# Specific group makeup tags

In [ ]:
# Extract specific group makeup tag IDs
specific_group_makeup_tag_ids = [tag["id"] for tag in group_makeup_specific_tags]

print(f"📊 Found {len(specific_group_makeup_tag_ids)} specific group makeup tags")

# Update the DataFrame to include specific tags analysis
scenes_data_specific = []
for scene in scenes_with_group_makeup_tags:
    scene_tags = [tag["id"] for tag in scene["tags"]]
    # Count how many specific group makeup tags this scene has
    specific_tags_count = sum(1 for tag_id in scene_tags if tag_id in specific_group_makeup_tag_ids)
    
    # Get only the group makeup tag names for this scene
    group_makeup_tag_names = [
        tag_id_to_name[tag_id] 
        for tag_id in scene_tags 
        if tag_id in all_group_makeup_tag_ids
    ]
    
    scenes_data_specific.append({
        "scene_id": scene["id"],
        "title": scene["title"],
        "specific_tags_count": specific_tags_count,
        "group_makeup_tags": group_makeup_tag_names
    })

df_scenes_specific = pl.DataFrame(scenes_data_specific)
df_scenes_specific


In [ ]:
# Find scenes with more than one specific group makeup tag
multiple_specific_tags_scenes = df_scenes_specific.filter(pl.col("specific_tags_count") > 1)

print(f"🎯 Found {len(multiple_specific_tags_scenes)} scenes with more than one specific group makeup tag")
print(f"📈 Out of {len(df_scenes_specific)} total scenes with group makeup tags")

# Show the distribution of specific tag counts
specific_tag_count_distribution = df_scenes_specific.group_by("specific_tags_count").agg(
    pl.len().alias("scene_count")
).sort("specific_tags_count")

print("\n📊 Distribution of specific group makeup tag counts:")
print(specific_tag_count_distribution)

# Display the scenes with multiple specific tags
if len(multiple_specific_tags_scenes) > 0:
    print(f"\n🔍 Scenes with multiple specific group makeup tags:")
    multiple_specific_tags_scenes.select(["scene_id", "title", "specific_tags_count"])


In [ ]:
# Show detailed tags for scenes with multiple specific group makeup tags
if len(multiple_specific_tags_scenes) > 0:
    print("🔍 Detailed analysis of scenes with multiple specific group makeup tags:")
    print("="*80)
    
    # Create a mapping of tag names to identify which are specific
    specific_tag_names = [tag["name"] for tag in group_makeup_specific_tags]
    
    for scene in multiple_specific_tags_scenes.iter_rows(named=True):
        scene_id = scene["scene_id"]
        title = scene["title"]
        group_makeup_tags = scene["group_makeup_tags"]
        
        # Find which specific tags this scene has
        specific_tags_for_scene = [tag_name for tag_name in group_makeup_tags if tag_name in specific_tag_names]
        
        print(f"\n📽️  Scene ID: {scene_id}")
        print(f"📋 Title: {title}")
        print(f"🏷️  All Group Makeup Tags ({len(group_makeup_tags)}):")
        for tag_name in group_makeup_tags:
            is_specific = tag_name in specific_tag_names
            prefix = "   🟡" if is_specific else "   🔵"
            print(f"{prefix} {tag_name}")
        print(f"🏷️  Specific Group Makeup Tags ({len(specific_tags_for_scene)}):")
        for tag_name in specific_tags_for_scene:
            print(f"   • {tag_name}")
        print("-" * 80)
        
else:
    print("✅ No scenes found with multiple specific group makeup tags!")


In [ ]:
# Apply the specific issue tag to scenes with multiple specific group makeup tags
if len(multiple_specific_tags_scenes) > 0:
    scene_ids_specific = multiple_specific_tags_scenes["scene_id"].to_list()
    result_specific = stash_client.bulk_scene_update(
        scene_ids_specific, 
        [stash_tags.data_quality_multiple_specific_group_makeup_tags["id"]], 
        "ADD"
    )
    print(f"✅ Applied 'Data Quality: Multiple Specific Group Makeup Tags' to {len(scene_ids_specific)} scenes")
    result_specific
else:
    print("✅ No scenes need the specific group makeup tags data quality tag")
